In [1]:
import os
os.chdir('../')
%pwd

'd:\\Text_Summarizer1'

In [2]:
from dataclasses import dataclass
from pathlib import Path 

@dataclass(frozen=True) 
class ModelEvaluationConfig:
    ''' return type --- these variable'''  
    root_dir: Path
    data_path: Path
    model_path: Path
    tokenizer_path: Path
    metric_file_name: Path

In [3]:
from textSummarizer.constants import *
from textSummarizer.utils.common import read_yaml, create_directories

In [4]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        config = self.config.model_trainer
        params = self.params.TrainingArguments

        create_directories([config.root_dir])

        model_evaluation_config = ModelEvaluationConfig(
            root_dir = config.root_dir,
            data_path = config.data_path,
            model_path = config.model_path,
            tokenizer_path = config.tokenizer_path ,
            metric_file_name = config.metric_file_name
        )

        return model_evaluation_config
    

    

In [5]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer 
from datasets import load_dataset, load_from_disk, load_metric
import torch 
import pandas as pd
from tqdm import tqdm 


In [6]:
class ModelEvaluation:
    def __init__(self, config: ModelEvaluationConfig):
        self.config = config 
        
    # Evaluation

    def generate_batch_sized_chunks(list_of_elements, batch_size):
        """split the dataset into smaller batches that we can process simultaneously
        Yield successive batch-sized chunks from list_of_elements."""
        for i in range(0, len(list_of_elements), batch_size):
            yield list_of_elements[i : i + batch_size]



    def calculate_metric_on_test_ds(self, dataset, metric, model, tokenizer,
                                batch_size=16, device='cuda',
                                column_text="article",
                                column_summary="highlights"):
        article_batches = list(self.generate_batch_sized_chunks(dataset[column_text], batch_size))
        target_batches = list(self.generate_batch_sized_chunks(dataset[column_summary], batch_size))

        for article_batch, target_batch in tqdm(
            zip(article_batches, target_batches), total=len(article_batches)):

            inputs = tokenizer(article_batch, max_length=1024,  truncation=True,
                            padding="max_length", return_tensors="pt")

            summaries = model.generate(input_ids=inputs["input_ids"].to(device),
                            attention_mask=inputs["attention_mask"].to(device),
                            length_penalty=0.8, num_beams=8, max_length=128)
            ''' parameter for length penalty ensures that the model does not generate sequences that are too long. '''

            # Finally, we decode the generated texts,
            # replace the  token, and add the decoded texts with the references to the metric.
            decoded_summaries = [tokenizer.decode(s, skip_special_tokens=True,
                                    clean_up_tokenization_spaces=True)
                for s in summaries]

            decoded_summaries = [d.replace("", " ") for d in decoded_summaries]


            metric.add_batch(predictions=decoded_summaries, references=target_batch)

        #  Finally compute and return the ROUGE scores.
        score = metric.compute()
        return score


    def evaluate(self):
        device = 'cuda' if torch.cuda.is_available() else 'cpu'
        #print("device available on stack ::", device)

        # loading the tokenizer and the model
        tokenizer = AutoTokenizer.from_pretrained(self.config.tokenizer_path) 
        model_pegasus = AutoModelForSeq2SeqLM.from_pretrained(self.config.model_path).to(device) 

        # loading the dataset which is converted in word embedding in data transformation phase
        dataset_samsum_pt = load_from_disk(self.config.data_path) 

        # metrics to calculate the SUMMARIZATION tasks
        rouge_names = ["rouge1", "rouge2", "rougeL", "rougeLsum"]
        rouge_metric = load_metric('rouge')

        score = self.calculate_metric_on_test_ds(
            dataset_samsum_pt['test'][0:10], rouge_metric, model_pegasus, tokenizer, batch_size = 2, device=device,
            column_text = 'dialogue', column_summary= 'summary'
        )

        rouge_dict = dict((rn, score[rn].mid.fmeasure) for rn in rouge_names)  

        # saving the result in pandas csv 
        df = pd.DataFrame(rouge_dict, index=['pegasus']) 
        df.to_csv(self.config.metric_file_name, index=False)



In [7]:
try:
    config = ConfigurationManager()
    model_evaluation_config = config.get_model_evaluation_config()
    model_evaluation_config = ModelTrainer(config=model_evaluation_config)
    model_evaluation_config.train()
    
except Exception as e:
    raise e

 in common file
{'artifacts_root': 'artifacts', 'data_ingestion': {'root_dir': 'artifacts/data_ingestion', 'source_URL': 'https://github.com/entbappy/Branching-tutorial/raw/master/summarizer-data.zip', 'local_data_file': 'artifacts/data_ingestion/data.zip', 'unzip_dir': 'artifacts/data_ingestion'}, 'data_validation': {'root_dir': 'artifacts/data_validation', 'STATUS_FILE': 'artifacts/data_validation/status.txt', 'ALL_REQUIRED_FILE': ['train', 'test', 'validation']}, 'data_transformation': {'root_dir': 'artifacts/data_transformation', 'data_path': 'artifacts/data_ingestion/samsum_dataset', 'tokenizer_name': 'google/pegasus-cnn_dailymail'}, 'model_trainer': {'root_dir': 'artifacts/model_trainer', 'data_path': 'artifacts/data_transformation/samsum_dataset', 'model_ckpt': 'google/pegasus-cnn_dailymail'}, 'model_evaluation': {'root_dir': 'artifacts/model_trainer', 'data_path': 'artifacts/data_transformation/samsum_dataset', 'model_path': 'artifacts/model_trainer/pegasus-samsum-model', 'toke

BoxKeyError: "'ConfigBox' object has no attribute 'model_path'"

In [15]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cpu'

In [20]:
import torch

print(torch.cuda.is_available())

False
